# Semantic Kernel

在此代码示例中，您将使用 [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI 框架创建一个基本 Agent。

此示例的目的是向您展示我们稍后在实现不同 Agentic 模式的其他代码示例中将使用的步骤。


## 导入所需的 Python 依赖包 

In [4]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv



from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

## 创建客户端 (Client) 和内核 (Kernel)

在此示例中，我们将使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 来访问 LLM。

`ai_model_id` 被定义为 `gpt-4o-mini`。尝试将模型更改为 GitHub Models 市场上可用的其他模型，以查看不同的结果。

为了使用 `Azure Inference SDK`（用于 GitHub Models 的 `base_url`），我们将在 Semantic Kernel 中使用 `AsyncOpenAI` 连接器。 还有其他[可用连接器](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)可将 Semantic Kernel 与其他模型提供商连接。

我们还将创建一个内核 `Kernel`。`kernel` 是您的 Agents 将使用的服务和插件的集合。在此代码段中，我们正在创建内核并将 `chat_completion_service` 添加到其中。


In [6]:
import random
from typing import Annotated
from semantic_kernel.functions import kernel_function

# 为示例定义一个示例插件

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # 度假目的地列表
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # 跟踪上一个目的地以避免重复
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # 获取可用的目的地（如果可能，排除上一个）
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # 选择一个随机的目的地
        destination = random.choice(available_destinations)

        # 更新上一个目的地
        self.last_destination = destination

        return destination


In [7]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")

kernel = Kernel()
kernel.add_plugin(DestinationsPlugin(), plugin_name="destinations")

service_id = "agent"

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id=service_id
)
kernel.add_service(chat_completion_service)

## 创建 Agent

下面我们将创建名为 `TravelAgent` 的 Agent，并创建一个名为 `AGENT_INSTRUCTIONS` 的变量。稍后我们会将其添加到我们的 `system_message` 中，该消息将向 Agent 提供有关任务、行为和语气的指示。

对于此示例，我们使用非常简单的指令。您可以更改这些指令以查看 Agent 如何做出不同的响应。


In [8]:
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [9]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = "You are a helpful AI Agent that can help plan vacations for customers at random destinations"
agent = ChatCompletionAgent(
    service_id=service_id, 
    kernel=kernel, 
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings)
)

## 运行 Agents

现在我们可以通过定义 `ChatHistory` 并将 `system_message` 添加到其中来运行 Agent。我们将使用之前定义的 `AGENT_INSTRUCTIONS`。

定义好这些之后，我们创建一个 `user_inputs`，它将是用户发送给 Agent 的内容。在这种情况下，我们将此消息设置为 `Plan me a sunny vacation`。

您可以随意更改此消息，以查看 Agent 如何做出不同的响应。


In [10]:
from IPython.display import display, HTML


async def main():
    # 定义聊天历史
    chat_history = ChatHistory()

    # 响应用户输入
    user_inputs = [
        "Plan me a day trip.",  # 计划一个一日游。
        "I don't like that destination. Plan me another vacation.",  # 我不喜欢那个目的地。为我计划另一个假期。
    ]

    for user_input in user_inputs:
        # 将用户输入添加到聊天历史
        chat_history.add_user_message(user_input)

        # 开始构建 HTML 输出
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"  # 用户：
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""
        function_calls = []
        function_results = {}

        # 收集 Agent 的响应并跟踪函数调用
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name

            # 跟踪函数调用和结果
            for item in content.items:
                if isinstance(item, FunctionCallContent):
                    call_info = f"Calling: {item.function_name}({item.arguments})"  # 正在调用：
                    function_calls.append(call_info)
                elif isinstance(item, FunctionResultContent):
                    result_info = f"Result: {item.result}"  # 结果：
                    function_calls.append(result_info)
                    # 存储函数结果
                    function_results[item.function_name] = item.result

            # 如果内容不是与函数相关的消息，则将其添加到响应中
            if (hasattr(content, 'content') and content.content and content.content.strip() and
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                        for item in content.items)):
                full_response += content.content

        # 如果发生任何函数调用，则将函数调用添加到 HTML
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"  # 函数调用（点击展开）
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"  # Increased font size and adjusted text color
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"

        # 将 Agent 响应添加到 HTML
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"  # 助手：
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # 显示格式化的 HTML
        display(HTML(html_output))

await main()
